In [ ]:
import pandas as pd
import numpy as np
from sympy import *
import openai
import re
import time
import random

In [ ]:
# assign variable value with your OpenAI key to continue
openai.api_key = ''

In [ ]:
# GPT-3
def generate_3(s):
    response = openai.Completion.create(
        model='text-davinci-003',
        prompt=s,
        temperature=0.1)
    return response['choices'][0]['text']

In [ ]:
# GPT-3.5
def generate_3_5(s):
    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[{
            "role": 'user',
            "content": s
        }],
        temperature=0.1
    )
    return completion.choices[0].message.content

In [ ]:
# GPT-4
def generate_4(s):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{
            "role": 'user',
            "content": s
        }],
        temperature=0.1)
    return response.choices[0].message.content

In [ ]:
# define variables
x = symbols("x")
y = symbols("y")
sym = " -> "

In [ ]:
# calculate accuracy of extracted equations
def acc_calculation(ans_df):
    corr_num = 0
    for i in range(len(ans_df)):
        # check if generated answer contains 2 equations and no characters other than x, y, digits and operators
        ans = ans_df.iloc[i, 1].replace(" ","")
        ans = ans.replace("\n","")
        ans = ans.rstrip('.')
        ans = ans.replace("^","**").split(";")
        invalid_char = re.findall(r'[^\^xy0-9=+-/\*;\(\) ]+', ans_df.iloc[i, 1])
        if "," in ans_df.iloc[i, 1]:
            continue
        elif len(ans)!=2 or len(invalid_char)!=0:
            continue 
        elif ans[1] == "":
            continue
        # proceed with valid answers
        # filling in asterisks
        temp = re.findall(r'[xy]|\d|[=+-/\*\(\)]', ans[0])
        j = 1
        while j<len(temp):
            if temp[j] == "x" or temp[j] == "y" or temp[j] == "(":
                if temp[j-1].isdigit() or temp[j-1] == ")" or temp[j-1] == "x" or temp[j-1] == "y":
                    temp.insert(j,"*")
                if temp[j-1] == ".":
                    temp.insert(j,"0*")
            elif temp[j].isdigit() and temp[j-1] == ")":
                temp.insert(j,"*")
            if temp[j] == "." and j+1 == len(temp):
                temp.pop(j)
            j +=1
        first = "".join(temp).split("=")
        j = 1
        temp = re.findall(r'[xy]|\d|[=+-/\*\(\)]', ans[1])
        while j<len(temp):
            if temp[j] == "x" or temp[j] == "y" or temp[j] == "(":
                if temp[j-1].isdigit() or temp[j-1] == ")" or temp[j-1] == "x" or temp[j-1] == "y":
                    temp.insert(j,"*")
                if temp[j-1] == ".":
                    temp.insert(j,"0*")
            elif temp[j].isdigit() and temp[j-1] == ")":
                temp.insert(j,"*")
            if temp[j] == "." and j+1 == len(temp):
                temp.pop(j)
            j +=1
        second = "".join(temp).split("=")
        if "" in first or "" in second:
            continue
        # create equations
        eq1 = Eq(eval(first[0]),eval(first[1]),evaluate=False)
        eq2 = Eq(eval(second[0]),eval(second[1]),evaluate=False)
        reference = ans_df.iloc[i, 0].split(";")
        one = reference[0].split("=")
        two = reference[1].split("=")
        ref1 = Eq(eval(one[0]),eval(one[1]),evaluate=False)
        ref2 = Eq(eval(two[0]),eval(two[1]),evaluate=False)
        # check if the model switched variables for representation
        temp = re.findall(r'[xy]|\d|[=+-/\*\(\)]', "=".join([first[0],first[1]]))
        k=0
        while k<len(temp):
            if temp[k] == "x":
                temp[k] = "y"
            elif temp[k] == "y":
                temp[k] = "x"
            k += 1
        sub1 = "".join(temp).split("=")
        temp = re.findall(r'[xy]|\d|[=+-/\*\(\)]', "=".join([second[0],second[1]]))
        k=0
        while k<len(temp):
            if temp[k] == "x":
                temp[k] = "y"
            elif temp[k] == "y":
                temp[k] = "x"
            k += 1
        sub2 = "".join(temp).split("=")
        se1 = Eq(eval(sub1[0]),eval(sub1[1]),evaluate=False)
        se2 = Eq(eval(sub2[0]),eval(sub2[1]),evaluate=False)
        # compare equations
        if solve(eq1)==solve(ref1) and solve(eq2)==solve(ref2):
            corr_num += 1
        elif solve(eq1)==solve(ref2) and solve(eq2)==solve(ref1):
            corr_num += 1
        elif solve(se1)==solve(ref1) and solve(se2)==solve(ref2):
            corr_num += 1
        elif solve(se1)==solve(ref2) and solve(se2)==solve(ref1):
            corr_num += 1
    return(corr_num/len(ans_df))

# Zero-shot Learning

In [ ]:
df100 = pd.read_excel('Data/L100.xlsx', header=None)
df100 = df100.rename(columns={0: "Question", 1: "Answer", 2: "Category"})
df100['ModelAnswer3'] = ''
df100['ModelAnswerTurbo'] = ''
df100['ModelAnswer4'] = ''

In [ ]:
# zero-shot 3
for index, row in df100.iterrows():
    s = 'Extract a system of two linear equations in terms of x and y from the following question. \
    Separate the equations with a semicolon.Do not explain what x and y represent, try solve the problem, \
    or add any explanations.\nQuestion: '+row['Question']+'\nEquations:'
    df100.loc[index, 'ModelAnswer3'] = generate_3(s)
    time.sleep(0.2)
print(acc_calculation(df100[['Answer', 'ModelAnswer3']]))

In [ ]:
# zero-shot 3.5
for index, row in df100.iterrows():
    s = 'Extract a system of two linear equations in terms of x and y from the following question. \
    Separate the equations with a semicolon.Do not explain what x and y represent, try solve the problem, \
    or add any explanations.\nQuestion: '+row['Question']+'\nEquations:'
    df100.loc[index, 'ModelAnswerTurbo'] = generate_3_5(s)
    time.sleep(0.2)

In [ ]:
print(acc_calculation(df100[['Answer', 'ModelAnswerTurbo']]))

In [ ]:
# zero-shot 4
for index, row in df100.iterrows():
    s = 'Extract a system of two linear equations in terms of x and y from the following question. \
    Separate the equations with a semicolon.Do not explain what x and y represent, try solve the problem, \
    or add any explanations. \nQuestion: '+row['Question']+'\nEquations:'
    df100.loc[index, 'ModelAnswer4'] = generate_4(s)
    time.sleep(0.2)

In [ ]:
print(acc_calculation(df100[['Answer', 'ModelAnswer4']]))

In [ ]:
df100.to_excel("ZeroShot.xlsx",index=False)

# Few-shot Learning

In [ ]:
# randomly draw sample questions from L1000
df1k = pd.read_excel('Data/L1000.xlsx', header=None)
df1k = df1k.rename(columns={0: "Question", 1: "Answer", 2: "Category"})
qs = pd.DataFrame(columns=['Question', 'Answer', 'Category'])
gp1k = df1k.groupby('Category')
for name, group in gp1k:
    temp = group.reset_index()
    idxs = random.sample(range(0,len(group)+1),1)
    for i in idxs:
        qs.loc[len(qs)] = [temp.loc[i, 'Question'], temp.loc[i, 'Answer'], name]

In [ ]:
# examples.xlsx contains examples we used
qs = pd.read_excel('examples.xlsx')

In [ ]:
# create the prompt
n = 5 # n is the number of examples giving to models, change value for different settings
s = "Extract a system of two linear equations in terms of x and y from the last question, like in the given examples. \
    Do not explain what x and y represent, try solve the problem, or add any explanations. \
    Do not use any other characters to represent the variables.\n"
for i in range(n):
    s = s+str(i+1)+'. '+qs.iloc[i,0]+sym+qs.iloc[i,1]+'\n'
s = s+str(n+1)+'. '

In [ ]:
# questions models extract equations from
df100 = pd.read_excel('Data/L100.xlsx', header=None)
df100 = df100.rename(columns={0: "Question", 1: "Answer", 2: "Category"})
df100['ModelAnswer3'] = ''
df100['ModelAnswerTurbo'] = ''
df100['ModelAnswer4'] = ''

In [ ]:
# GPT-3
for index, row in df100.iterrows():
    resp = generate_3(s+row['Question']+sym)
    df100.loc[index, 'ModelAnswer3'] = resp.replace("\n", "")

# GPT-3.5
for index, row in df100.iterrows():
    resp = generate_3_5(s+row['Question']+sym)
    df100.loc[index, 'ModelAnswerTurbo'] = resp.replace("\n", "")
    time.sleep(0.2)

# GPT-4
for index, row in df100.iterrows():
    resp = generate_4(s+row['Question']+sym)
    df100.loc[index, 'ModelAnswer4'] = resp.replace("\n", "")

In [ ]:
print(acc_calculation(df100[['Answer', 'ModelAnswer3']]))
print(acc_calculation(df100[['Answer', 'ModelAnswerTurbo']]))
print(acc_calculation(df100[['Answer', 'ModelAnswer4']]))
#df100.to_excel("5ShotT1.xlsx",index=False)

In [ ]:
# Accuracy by category GPT-3
for name, group in df100.groupby('Category'):
    print(name)
    print(acc_calculation(group[['Answer', 'ModelAnswer3']]))

In [ ]:
# Accuracy by category GPT-3.5
for name, group in df100.groupby('Category'):
    print(name)
    print(acc_calculation(group[['Answer', 'ModelAnswerTurbo']]))

In [ ]:
# Accuracy by category GPT-4
for name, group in df100.groupby('Category'):
    print(name)
    print(acc_calculation(group[['Answer', 'ModelAnswer4']]))

# Fine-tuning
#### Fine-tuned model is not accessible for other accounts. Follow the fine-tuning guidelines and use L1000 to reproduce.